In [1]:
import numpy as np
import pandas as pd
import cvxpy as cp

# (a) Optimum of LS problem
Let $g(x)=||\tilde{y}-y||_2^2$, then we have $\nabla g(x)=-2H^T\tilde{y}-Hx=0$ and $\nabla^2 g(x)=2H^TH>0$, which leads to the solution of question: 

$$ \hat{x}=(H^TH)^{-1}H^T\tilde{y} $$

which is the optimum of the solution.

# (b) return the matrix of H

In [2]:
def matrix_return(t,theta):
    H = []
    for i in range(len(theta)):
        H.append(np.cos(theta[i]*t))
    return np.transpose(H)

# (c)
## Aronson’s sequence

In [48]:
y_tilde_data = pd.read_csv('mathias_distress_call_1.csv')
y_tilde = y_tilde_data.values
y_tilde_data_2 = pd.read_csv('mathias_distress_call_2.csv')
y_tilde_2 = y_tilde_data_2.values

dt = 1.0/8192
t = np.arange(0,len(y_tilde)*dt,dt)
theta = [104, 111, 116, 122, 126, 131, 136, 142, 147, 158, 164, 169, 
         174, 181, 183, 193, 199, 205, 208, 214, 220, 226, 231, 237, 
         243, 249, 254]

H = matrix_return(t,theta)
x_hat_i = np.linalg.solve(np.dot(np.transpose(H),H),np.dot(np.transpose(H),y_tilde))
print('residual error:', np.linalg.norm(y_tilde-np.dot(H,x_hat_i)))

residual error: 2046.298989299373


## Mathias own musical scale

In [49]:
theta = np.arange(311.127, 311.127+311.127,311.127/27)
H = matrix_return(t,theta)
x_hat_ii = np.linalg.solve(np.dot(np.transpose(H),H),np.dot(np.transpose(H),y_tilde))
print('residual error:', np.linalg.norm(y_tilde-np.dot(H,x_hat_ii)))

residual error: 1963.530567863447


## "Fibonacci-on-steroids" sequence

In [50]:
theta = np.zeros(27)
theta[0] = 150
theta[1] = 175
for i in range(25):
    theta[i+2] = np.ceil(0.5*theta[i+1])+np.ceil(0.8*theta[i])
H = matrix_return(t,theta)
x_hat_iii = np.linalg.solve(np.dot(np.transpose(H),H),np.dot(np.transpose(H),y_tilde))
print('residual error:', np.linalg.norm(y_tilde_2-np.dot(H,x_hat_iii)))

residual error: 9675.046427897283


Based on the residual error, we know "Fibonacci-on-steroids" sequences works the best. We find from the solution that some values are very close to zero. We round the solution and find that this solution is sparse and only $a_5$, $a_7$, $a_8$, $a_{18}$, $a_{19}$, $a_{20}$, $a_{21}$, $a_{27}$ are nonzero, these values are printed below. Therefore, the message only contains the frequency of $\theta_5$, $\theta_7$, $\theta_8$, $\theta_{18}$, $\theta_{19}$, $\theta_{20}$, $\theta_{21}$, $\theta_{27}$

In [51]:
round_ = np.rint(x_hat_iii)
print('(integer) amplitudes of existing frequencies:',round_[round_.nonzero()])

(integer) amplitudes of existing frequencies: [2. 6. 3. 4. 1. 7. 6. 1.]


# (d)
Based on encoding rules, and to make the message meaningful, I believe the message is "Sehr Gut"

# (e) With CVXPY package and Lasso

## Aronson's sequence

In [55]:
theta = [104, 111, 116, 122, 126, 131, 136, 142, 147, 158, 164, 169, 
         174, 181, 183, 193, 199, 205, 208, 214, 220, 226, 231, 237, 
         243, 249, 254]

H = matrix_return(t,theta)

lambd_value = 0.0001
x_cp = cp.Variable(27)
obj = cp.Minimize(cp.norm(y_tilde_2[:,0] - H @ x_cp)**2 + lambd_value * cp.norm(x_cp,1))

prob = cp.Problem(obj)
prob.solve(solver=cp.SCS)
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x_cp.value)
print('residual error:', np.linalg.norm(y_tilde_2-np.dot(H,np.array(x_cp.value).reshape(27,1))))

status: optimal_inaccurate
optimal value 242100.42135340036
optimal var [-2.63475642e-04 -1.10878818e-05  8.94272346e-05  1.21330467e-04
 -2.11681481e-04  3.36793611e-06 -1.64404896e-04 -2.95901410e-04
 -2.49812762e-04 -7.12413489e-04 -6.97624608e-04 -3.87984850e-04
 -1.04548216e-04  1.21288218e-04 -2.58514165e-04  1.22892646e-04
 -7.54110207e-04  9.65737219e-06 -6.56118752e-04 -6.15890669e-04
 -1.14241774e-07  9.69320464e-05 -1.53439917e-04 -5.68292955e-04
 -1.33910299e-04 -1.90229735e-04 -3.23304007e-04]
residual error: 9523.015065991236


## Mathias own musical scale

In [ ]:
theta = np.arange(311.127, 311.127+311.127,311.127/27)
H = matrix_return(t,theta)

lambd_value = 0.00001
x_cp = cp.Variable(27, integer=True)
obj = cp.Minimize(cp.norm(y_tilde_2[:,0] - H @ x_cp)**2 + lambd_value * cp.norm(x_cp,1))

prob = cp.Problem(obj, [x_cp>=1])
prob.solve(solver=cp.MOSEK)
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x_cp.value)
print('residual error:', np.linalg.norm(y_tilde_2-np.dot(H,np.array(x_cp.value).reshape(27,1))))

## "Fibonacci-on-steroids" sequence

In [44]:
theta = np.zeros(27)
theta[0] = 150
theta[1] = 175
for i in range(25):
    theta[i+2] = np.ceil(0.5*theta[i+1])+np.ceil(0.8*theta[i])
H = matrix_return(t,theta)

lambd_value = 0.001
x_cp = cp.Variable(27)
obj = cp.Minimize(cp.norm(y_tilde_2[:,0] - H @ x_cp)**2 + lambd_value * cp.norm(x_cp,1))

prob = cp.Problem(obj)
prob.solve(solver=cp.SCS)
print("status:", prob.status)
print("optimal value", prob.value)
print("optimal var", x_cp.value)
print('residual error:', np.linalg.norm(y_tilde_2-np.dot(H,np.array(x_cp.value).reshape(27,1))))

array([[-0.33173347],
       [ 0.05680628],
       [-0.42516628],
       [ 0.13762933],
       [ 0.1439599 ],
       [ 0.06582349],
       [ 0.35031757],
       [ 0.22929124],
       [-0.40681551],
       [-0.0167118 ],
       [-0.06901476],
       [-0.17244492],
       [ 0.27471442],
       [ 0.20630256],
       [ 0.36050138],
       [ 0.17735724],
       [ 0.16229146],
       [ 0.31889703],
       [ 0.27750422],
       [ 0.0695776 ],
       [ 0.11053446],
       [ 0.08402851],
       [ 0.14467749],
       [-0.13855879],
       [ 0.06976102],
       [ 0.19401025],
       [ 0.00589049]])

In [43]:
x_hat_iii.shape

(27, 1)